In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cartopy
from visjobs.visualize import easy_plot
from cartopy.io.img_tiles import Stamen, StamenTerrain, QuadtreeTiles, GoogleWTS, OSM, MapQuestOpenAerial
import matplotlib.patheffects as PathEffects
import os
import scipy.constants as sc


from pylab import rcParams
rcParams['figure.figsize'] = 21,24

In [23]:
%%capture

def plot_quantile_map(ds, var, title_max, title_min, title_ave):
    # görsel


    #draw map
    m = easy_plot.painter()
    tiler = Stamen('watercolor')

    for i in range(len(ds)):

        if i == 0:
            short_title = short_title_min
            saved_method = 'min'

        elif i == 1:
            short_title = short_title_max
            saved_method = 'max'

        elif i == 2:
            short_title = short_title_ave
            saved_method = 'ave'
        
        lon = ds[0]['longitude']
        lat = ds[0]['latitude']

        work_title = '{}'.format(dates,)

        #paint features
        ax = m.paint_ax(1,1,1, check_proj=True)

        #features
        ax.add_feature(cartopy.feature.LAND.with_scale('10m'),  zorder=1, linewidths=1.5, edgecolor='none', facecolor='white')
        m.paint_borders(ax=ax, res='10m', zorder=4, linewidths=1.5, edgecolor='red' )
        m.paint_lakes(ax=ax, res='10m', zorder=2, linewidths=1.5, edgecolor='red', facecolor='none' )
        m.paint_states(ax=ax, res='10m', zorder=4, linewidths=0.4, edgecolor='red',  )
        m.paint_coastline(ax=ax, res='10m', zorder=3, linewidths=1.8, edgecolor='red')

        ax.set_extent([25, 45, 33, 45])
        
        if var == '500height':
            interval = np.arange(np.min(ds[i]), np.max(ds[i]), 6)
            
        if var == '2mtemp':
            interval = np.arange(np.min(ds[i]), np.max(ds[i]), 2)
            
        if var == '2mdew':
            interval = np.arange(np.min(ds[i]), np.max(ds[i]), 2)
            
        if var == 'msl':
            interval = np.arange(np.min(ds[i]), np.max(ds[i]), 1)
        
        #make the freezing rain reflectivities contourf
        mesh = ax.contour(lon, lat, ds[i], interval, transform=cartopy.crs.PlateCarree(), zorder=6, alpha=0.9,
                            colors='k' )
        m.plot_clabel(mesh, fontsize=25, inline=1, inline_spacing=7,fmt='%i', rightside_up=True, use_clabeltext=True , ax=ax, zorder=5)


        valid_era = m.set_title(title='ERA-5 Reanalysis', right = 0.850, up=0.010, ax=ax, fontsize=16,
                                     style='italic', color='black', zorder=20, transform=ax.transAxes, weight='bold'
                                     )

        valid_t = m.set_title(title='1991-2020 Baseline', right = 0.850, up=1.010, ax=ax, fontsize=16,
                                     style='italic', color='red', zorder=20, transform=ax.transAxes, weight='bold'
                                     )

        #titles
        title_short = m.set_title(title='{}'.format(short_title), ax=ax, fontsize=20, up=1.030, 
                                 weight='heavy',style='italic',transform=ax.transAxes)

        title1 = m.set_title(title='{}'.format(work_title), ax=ax, fontsize=16, up=1.010, 
                                 weight='heavy',style='italic',transform=ax.transAxes)
        title2 = m.set_title(title='Climaturk.com', color='white', right=0.00730, up=0.9615000, ax=ax, size=25, 
                                     zorder=53,style='italic',transform=ax.transAxes, weight='bold',
                                        )
        title2.set_path_effects([PathEffects.withStroke(linewidth=6, foreground='k')])
        
        # savefig
        #storm için yeni data klasörü oluştur
        os.chdir(r'D:\JupyterLab\Climaturk_Site\Docs\STORMS\Quantiles')
        try:
            os.mkdir('{}'.format(str(DateList[count, 0])))
        except:
            print('Directory exists')
        
        #save figure
        plt.savefig(r'D:\JupyterLab\Climaturk_Site\Docs\STORMS\Quantiles\{}\{}_{}.jpeg'.format(str(DateList[count, 0]),
                                                                                                          var,
                                                                                                          saved_method),
                        bbox_inches='tight', optimize=True, progressive=True, dpi=150,
                        )
        plt.close()

In [18]:
def adjust_data_units(pressure_data, single_data):
    
    pressure_data['z'] = np.divide(pressure_data['z'], sc.g) # geop. to height
    
    single_data['t2m'] = single_data['t2m'] - 273.15 # K to C
    single_data['d2m'] = single_data['d2m'] - 273.15 # K to C
    single_data['msl'] = single_data['msl'] / 100 # Pa to hPa
    
    return pressure_data, single_data

In [19]:
def find_min_max_ave(data, var):
    
    return data[var].min(dim='time'), \
           data[var].max(dim='time'), \
           data[var].mean(dim='time')

In [25]:
DateList = np.load(r'C:\Users\USER\JupyterLab\Climaturk_Site\GITHUB\version_17_05_2021\Climaturk-Base\src/DateList_Quantile_10042021.npy', allow_pickle=True)

# Define the path
#storm klasörüne geçiş yap
for count in range(len(DateList)):
    
    print('THE DATES IS: ', str(DateList[count, 0]))
    
    # If the date has already processed, than continue to next loop. first cond
    # If the plot already done continue to next loop. second cond
    if DateList[count, 2] == 0 or DateList[count, 3] == 1: continue

    dates = str(DateList[count, 0])
    #dates_s =  str(DateList[count, 1])
    
    
    pre_data_path = r'D:\JupyterLab\Climaturk_Site\Docs\STORM_DATA'
    full_path_pr = pre_data_path + '\\' + dates + '\\' + 'quantile_pressure_data.nc'
    full_path_sn = pre_data_path + '\\' + dates + '\\' + 'quantile_single_data.nc'

    pressure_data = xr.open_dataset(full_path_pr)
    single_data = xr.open_dataset(full_path_sn)
    
    #adjust dataset units
    adjusted_pressure, adjusted_single = adjust_data_units(pressure_data, single_data)
    
    # min - max - ave for z
    zmin_daily, zmax_daily, zave_daily = find_min_max_ave(adjusted_pressure, 'z')
    
    # min - max - ave for t
    tmin_daily, tmax_daily, tave_daily = find_min_max_ave(adjusted_single, 't2m')
    
    # min - max - ave for d
    dmin_daily, dmax_daily, dave_daily = find_min_max_ave(adjusted_single, 'd2m')
    
    # min - max - ave for p
    pmin_daily, pmax_daily, pave_daily = find_min_max_ave(adjusted_single, 'msl')

    ds_z = [zmin_daily, zmax_daily, zave_daily]
    ds_t = [tmin_daily, tmax_daily, tave_daily]    
    ds_d = [dmin_daily, dmax_daily, dave_daily]    
    ds_p = [pmin_daily, pmax_daily, pave_daily]
    
    
    # PLOT
    # z
    var = '500height'
    short_title_max = '500 mb Maximum Geopotential Height (m)'
    short_title_min = '500 mb Minimum Geopotential Height (m)'
    short_title_ave = '500 mb Average Geopotential Height (m)'

    plot_quantile_map(ds_z, var, short_title_min, short_title_max, short_title_ave, 
                     )
    print(f'exitted Plotting: {var}')
    
    # t
    var = '2mtemp'
    short_title_max = '2m  Maximum Temperature (degC)'
    short_title_min = '2m  Minimum Temperature (degC)'
    short_title_ave = '2m  Average Temperature (degC)'

    plot_quantile_map(ds_t, var, short_title_min, short_title_max, short_title_ave, 
                     )
    print(f'exitted Plotting: {var}')
    
    # d
    var = '2mdew'
    short_title_max = '2m  Maximum Dewpoint Temperature (degC)'
    short_title_min = '2m  Minimum Dewpoint Temperature (degC)'
    short_title_ave = '2m  Average Dewpoint Temperature (degC)'

    plot_quantile_map(ds_d, var, short_title_min, short_title_max, short_title_ave, 
                     )
    print(f'exitted Plotting: {var}')
    
    # p
    var = 'msl'
    short_title_max = 'Maximum Mean Sea Level Pressure (mb)'
    short_title_min = 'Minimum Mean Sea Level Pressure (mb)'
    short_title_ave = 'Average Mean Sea Level Pressure (mb)'

    plot_quantile_map(ds_p, var, short_title_min, short_title_max, short_title_ave, 
                     )
    print(f'exitted Plotting: {var}')
    
    # change 0 to 1 in corresponding plot's [done] column (means the plot is completed in plotting)
    DateList[count, 3] = 1
    np.save(r'C:\Users\USER\JupyterLab\Climaturk_Site\GITHUB\version_17_05_2021\Climaturk-Base\src/DateList_Quantile_10042021', DateList)
    print('Plot Done: ', str(DateList[count, 0]))
    
    break

THE DATES IS:  20141212
Directory exists


<ipython-input-23-66d9ed65f916>:86: MatplotlibDeprecationWarning: 
The 'optimize' parameter of print_jpg() was deprecated in Matplotlib 3.3 and will be removed two minor releases later. Use pil_kwargs={'optimize': ...} instead. If any parameter follows 'optimize', they should be passed as keyword, not positionally.
  plt.savefig(r'D:\JupyterLab\Climaturk_Site\Docs\STORMS\Quantiles\{}\{}_{}.jpeg'.format(str(DateList[count, 0]),
<ipython-input-23-66d9ed65f916>:86: MatplotlibDeprecationWarning: 
The 'progressive' parameter of print_jpg() was deprecated in Matplotlib 3.3 and will be removed two minor releases later. Use pil_kwargs={'progressive': ...} instead. If any parameter follows 'progressive', they should be passed as keyword, not positionally.
  plt.savefig(r'D:\JupyterLab\Climaturk_Site\Docs\STORMS\Quantiles\{}\{}_{}.jpeg'.format(str(DateList[count, 0]),


Directory exists


<ipython-input-23-66d9ed65f916>:86: MatplotlibDeprecationWarning: 
The 'optimize' parameter of print_jpg() was deprecated in Matplotlib 3.3 and will be removed two minor releases later. Use pil_kwargs={'optimize': ...} instead. If any parameter follows 'optimize', they should be passed as keyword, not positionally.
  plt.savefig(r'D:\JupyterLab\Climaturk_Site\Docs\STORMS\Quantiles\{}\{}_{}.jpeg'.format(str(DateList[count, 0]),
<ipython-input-23-66d9ed65f916>:86: MatplotlibDeprecationWarning: 
The 'progressive' parameter of print_jpg() was deprecated in Matplotlib 3.3 and will be removed two minor releases later. Use pil_kwargs={'progressive': ...} instead. If any parameter follows 'progressive', they should be passed as keyword, not positionally.
  plt.savefig(r'D:\JupyterLab\Climaturk_Site\Docs\STORMS\Quantiles\{}\{}_{}.jpeg'.format(str(DateList[count, 0]),


Directory exists


...

Directory exists


<ipython-input-23-66d9ed65f916>:86: MatplotlibDeprecationWarning: 
The 'optimize' parameter of print_jpg() was deprecated in Matplotlib 3.3 and will be removed two minor releases later. Use pil_kwargs={'optimize': ...} instead. If any parameter follows 'optimize', they should be passed as keyword, not positionally.
  plt.savefig(r'D:\JupyterLab\Climaturk_Site\Docs\STORMS\Quantiles\{}\{}_{}.jpeg'.format(str(DateList[count, 0]),
<ipython-input-23-66d9ed65f916>:86: MatplotlibDeprecationWarning: 
The 'progressive' parameter of print_jpg() was deprecated in Matplotlib 3.3 and will be removed two minor releases later. Use pil_kwargs={'progressive': ...} instead. If any parameter follows 'progressive', they should be passed as keyword, not positionally.
  plt.savefig(r'D:\JupyterLab\Climaturk_Site\Docs\STORMS\Quantiles\{}\{}_{}.jpeg'.format(str(DateList[count, 0]),


Directory exists


<ipython-input-23-66d9ed65f916>:86: MatplotlibDeprecationWarning: 
The 'optimize' parameter of print_jpg() was deprecated in Matplotlib 3.3 and will be removed two minor releases later. Use pil_kwargs={'optimize': ...} instead. If any parameter follows 'optimize', they should be passed as keyword, not positionally.
  plt.savefig(r'D:\JupyterLab\Climaturk_Site\Docs\STORMS\Quantiles\{}\{}_{}.jpeg'.format(str(DateList[count, 0]),
<ipython-input-23-66d9ed65f916>:86: MatplotlibDeprecationWarning: 
The 'progressive' parameter of print_jpg() was deprecated in Matplotlib 3.3 and will be removed two minor releases later. Use pil_kwargs={'progressive': ...} instead. If any parameter follows 'progressive', they should be passed as keyword, not positionally.
  plt.savefig(r'D:\JupyterLab\Climaturk_Site\Docs\STORMS\Quantiles\{}\{}_{}.jpeg'.format(str(DateList[count, 0]),


exitted Plotting: msl
Plot Done:  20141212


In [81]:
%%capture

var = '500height'
short_title_max = '500 mb Hourly Maximum Geopotential Height'
short_title_min = '500 mb Hourly Minimum Geopotential Height'
short_title_ave = '500 mb Hourly Average Geopotential Height'

plot_quantile_map(ds_z, var, short_title_min, short_title_max, short_title_ave, 
                 )

Directory exists


<ipython-input-80-dd54d2a17bc2>:77: MatplotlibDeprecationWarning: 
The 'optimize' parameter of print_jpg() was deprecated in Matplotlib 3.3 and will be removed two minor releases later. Use pil_kwargs={'optimize': ...} instead. If any parameter follows 'optimize', they should be passed as keyword, not positionally.
  plt.savefig(r'C:\Users\USER\JupyterLab\Climaturk_Site\Docs\STORMS\Quantiles\{}\{}_{}.jpeg'.format(str(DateList[count, 0]),
<ipython-input-80-dd54d2a17bc2>:77: MatplotlibDeprecationWarning: 
The 'progressive' parameter of print_jpg() was deprecated in Matplotlib 3.3 and will be removed two minor releases later. Use pil_kwargs={'progressive': ...} instead. If any parameter follows 'progressive', they should be passed as keyword, not positionally.
  plt.savefig(r'C:\Users\USER\JupyterLab\Climaturk_Site\Docs\STORMS\Quantiles\{}\{}_{}.jpeg'.format(str(DateList[count, 0]),


Directory exists


<ipython-input-80-dd54d2a17bc2>:77: MatplotlibDeprecationWarning: 
The 'optimize' parameter of print_jpg() was deprecated in Matplotlib 3.3 and will be removed two minor releases later. Use pil_kwargs={'optimize': ...} instead. If any parameter follows 'optimize', they should be passed as keyword, not positionally.
  plt.savefig(r'C:\Users\USER\JupyterLab\Climaturk_Site\Docs\STORMS\Quantiles\{}\{}_{}.jpeg'.format(str(DateList[count, 0]),
<ipython-input-80-dd54d2a17bc2>:77: MatplotlibDeprecationWarning: 
The 'progressive' parameter of print_jpg() was deprecated in Matplotlib 3.3 and will be removed two minor releases later. Use pil_kwargs={'progressive': ...} instead. If any parameter follows 'progressive', they should be passed as keyword, not positionally.
  plt.savefig(r'C:\Users\USER\JupyterLab\Climaturk_Site\Docs\STORMS\Quantiles\{}\{}_{}.jpeg'.format(str(DateList[count, 0]),


Directory exists


<ipython-input-80-dd54d2a17bc2>:77: MatplotlibDeprecationWarning: 
The 'optimize' parameter of print_jpg() was deprecated in Matplotlib 3.3 and will be removed two minor releases later. Use pil_kwargs={'optimize': ...} instead. If any parameter follows 'optimize', they should be passed as keyword, not positionally.
  plt.savefig(r'C:\Users\USER\JupyterLab\Climaturk_Site\Docs\STORMS\Quantiles\{}\{}_{}.jpeg'.format(str(DateList[count, 0]),
<ipython-input-80-dd54d2a17bc2>:77: MatplotlibDeprecationWarning: 
The 'progressive' parameter of print_jpg() was deprecated in Matplotlib 3.3 and will be removed two minor releases later. Use pil_kwargs={'progressive': ...} instead. If any parameter follows 'progressive', they should be passed as keyword, not positionally.
  plt.savefig(r'C:\Users\USER\JupyterLab\Climaturk_Site\Docs\STORMS\Quantiles\{}\{}_{}.jpeg'.format(str(DateList[count, 0]),
